### set device and data

In [1]:
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'using device: {DEVICE}')

using device: cuda


In [2]:
def file2list(file):
    lst = []
    for line in open(f'./{file}', 'r', encoding='utf-8-sig'):
      line = line.replace('\n', '')
      lst.append(line)
    return lst

# train, valid, test texts
en_train_lst = file2list('train.en')
en_valid_lst = file2list('val.en')
en_test_lst  = file2list('test.en')
print(f'en data: {len(en_train_lst), len(en_valid_lst), len(en_test_lst)}')

de_train_lst = file2list('train.de')
de_valid_lst = file2list('val.de')
de_test_lst  = file2list('test.de')
print(f'de data: {len(de_train_lst), len(de_valid_lst), len(de_test_lst)}')

en data: (29001, 1015, 1000)
de data: (29001, 1015, 1000)


### build vocabularies

In [3]:
# using spacy tokenizer
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 26.7 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.6 MB 26.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [4]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# using spacy tokenizer
en_tokenizer = get_tokenizer(tokenizer='spacy', language='en_core_web_sm')
de_tokenizer = get_tokenizer(tokenizer='spacy', language='de_core_news_sm')

# vocabulary for training texts
en_vocab = build_vocab_from_iterator(map(en_tokenizer, [text for text in en_train_lst]),
                                     min_freq=2,
                                     specials=['<unk>','<sos>','<eos>','<pad>'],
                                     special_first=True)
de_vocab = build_vocab_from_iterator(map(de_tokenizer, [text for text in de_train_lst]),
                                     min_freq=2,
                                     specials=['<unk>','<sos>','<eos>','<pad>'],
                                     special_first=True)
print(f'en vocab size: {len(en_vocab)}')
print(f'de vocab size: {len(de_vocab)}')

en vocab size: 6191
de vocab size: 8014


### make preprocessor for encode & decode texts

In [5]:
class Preprocessor:
  unk_token_id = 0
  sos_token_id = 1
  eos_token_id = 2
  pad_token_id = 3

  def __init__(self, src_tokenizer, tgt_tokenizer, src_vocab, tgt_vocab):
    self.src_tokenizer = src_tokenizer
    self.tgt_tokenizer = tgt_tokenizer

    # token -> id
    self.src_token2id = src_vocab.get_stoi()
    self.tgt_token2id = tgt_vocab.get_stoi()
    # id -> token
    self.src_id2token = src_vocab.get_itos()
    self.tgt_id2token = tgt_vocab.get_itos()

  # encode token -> id for source sentence
  def src_encode(self, text):
    if type(text) == list: text = ' '.join(text)
    tokenized = self.src_tokenizer(text)
    encoded   = [self.src_token2id.get(token, self.src_token2id['<unk>']) for token in tokenized]
    return encoded

  # encode token -> id, attach <sos> and <eos> for target sentence
  def tgt_encode(self, text):
    if type(text) == list: text = ' '.join(text)
    tokenized = self.tgt_tokenizer(text)
    encoded   = [self.tgt_token2id['<sos>']] \
    + [self.tgt_token2id.get(token, self.tgt_token2id['<unk>']) for token in tokenized] \
    + [self.tgt_token2id['<eos>']]
    return encoded

  # decode source sentence
  def src_decode(self, ids):
    decoded = list(map(lambda x: self.src_id2token[x], ids))
    return ' '.join(decoded)

  # decode target sentence
  def tgt_decode(self, ids):
    decoded = list(map(lambda x: self.tgt_id2token[x], ids))[1:-1]
    return ' '.join(decoded)

preprocessor = Preprocessor(en_tokenizer, de_tokenizer, en_vocab, de_vocab)

### make custom dataset, data loader

In [6]:
from torch.utils.data.dataset import Dataset

class CustomDataset(Dataset):
  def __init__(self, en_lst, de_lst, preprocessor):
    self.en_lst = en_lst
    self.de_lst = de_lst
    assert len(self.en_lst) == len(self.de_lst)

    self.preprocessor = preprocessor
    self.dataset      = self.make_dataset()

  # make (en, de) sentence pair dataset
  def make_dataset(self):
    dataset = [(self.preprocessor.src_encode(en), self.preprocessor.tgt_encode(de))
               for en, de in zip(self.en_lst, self.de_lst)
               if len(en) > 0 and len(de) > 0]
    return dataset

  def __getitem__(self, idx):
    return self.dataset[idx]

  def __len__(self):
    return len(self.dataset)

train_dataset = CustomDataset(en_train_lst, de_train_lst, preprocessor)
valid_dataset = CustomDataset(en_valid_lst, de_valid_lst, preprocessor)
test_dataset  = CustomDataset(en_test_lst, de_test_lst, preprocessor)
print(f'dataset size: {len(train_dataset), len(valid_dataset), len(test_dataset)}')

dataset size: (29000, 1014, 1000)


In [7]:
import random
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

# sequence padding in batch
def collate_fn(batch_samples):
  pad_token_id = preprocessor.pad_token_id
  src_sent = pad_sequence([torch.tensor(src) for src, _ in batch_samples],
                          batch_first=True,
                          padding_value=pad_token_id)
  tgt_sent = pad_sequence([torch.tensor(tgt) for _, tgt in batch_samples],
                          batch_first=True,
                          padding_value=pad_token_id)
  return src_sent.to(DEVICE), tgt_sent.to(DEVICE)

# composing batch to match similar lengths of sequences
def batch_sampling(sequence_lengths, BATCH_SIZE):
  seq_lens = [(i, seq_len, tgt_len) for i, (seq_len, tgt_len) in enumerate(sequence_lengths)]
  seq_lens = sorted(seq_lens, key=lambda x: x[1])
  seq_lens = [sample[0] for sample in seq_lens]

  sample_indices = [seq_lens[i:i+BATCH_SIZE] for i in range(0, len(seq_lens), BATCH_SIZE)]
  random.shuffle(sample_indices)
  return sample_indices

# make dataloader using sequence padding & composing batch
def make_loader(dataset, BATCH_SIZE):
  sequence_lengths = list(map(lambda x: (len(x[0]), len(x[1])), dataset))
  batch_sampler = batch_sampling(sequence_lengths, BATCH_SIZE)
  loader = DataLoader(dataset,
                      collate_fn=collate_fn,
                      batch_sampler=batch_sampler)
  return loader

BATCH_SIZE = 64
train_loader = make_loader(train_dataset, BATCH_SIZE)
valid_loader = make_loader(valid_dataset, BATCH_SIZE)
test_loader  = make_loader(test_dataset, BATCH_SIZE)
print('check dataset and batch size,')

print(f'train data length       : {len(en_train_lst)}')
print(f'train data loader length: {len(train_loader)}')
print(f'train data loader length: {len(en_train_lst) / BATCH_SIZE}\n')

print(f'valid data length       : {len(en_valid_lst)}')
print(f'valid data loader length: {len(valid_loader)}')
print(f'valid data loader length: {len(en_valid_lst) / BATCH_SIZE}\n')

print(f'test data length       : {len(en_test_lst)}')
print(f'test data loader length: {len(test_loader)}')
print(f'test data loader length: {len(en_test_lst) / BATCH_SIZE}')

check dataset and batch size,
train data length       : 29001
train data loader length: 454
train data loader length: 453.140625

valid data length       : 1015
valid data loader length: 16
valid data loader length: 15.859375

test data length       : 1000
test data loader length: 16
test data loader length: 15.625


### multi-head attention, position-wise feed forward

In [8]:
import torch.nn as nn

class MultiHeadAttentionLayer(nn.Module):
  def __init__(self, HIDDEN_DIM, HEADS, dropout_ratio):
    super().__init__()
    assert HIDDEN_DIM % HEADS == 0
    self.HIDDEN_DIM = HIDDEN_DIM  
    self.HEADS      = HEADS
    self.HEAD_DIM   = HIDDEN_DIM // HEADS  # n_heads * hidden_dim = head_dim

    self.fc_query = nn.Linear(HIDDEN_DIM, HIDDEN_DIM)
    self.fc_key   = nn.Linear(HIDDEN_DIM, HIDDEN_DIM)
    self.fc_value = nn.Linear(HIDDEN_DIM, HIDDEN_DIM)
    self.fc_o     = nn.Linear(HIDDEN_DIM, HIDDEN_DIM)

    self.SCALE   = torch.sqrt(torch.FloatTensor([self.HEAD_DIM])).to(DEVICE)
    self.dropout = nn.Dropout(dropout_ratio)

  def forward(self, query, key, value, mask=None):
    batch_size = query.shape[0]
    # query: [batch_size, query_len, hidden_dim]
    # key  : [batch_size, key_len, hidden_dim]
    # value: [batch_size, value_len, hidden_dim]

    Q = self.fc_query(query)
    K = self.fc_key(key)
    V = self.fc_value(value)
    # Q: [batch_size, query_len, hidden_dim]
    # K: [batch_size, key_len, hidden_dim]
    # V: [batch_size, value_len, hidden_dim]

    # hidden_dim to n_heads * head_dim
    Q = Q.view(batch_size, -1, self.HEADS, self.HEAD_DIM).permute(0, 2, 1, 3)
    K = K.view(batch_size, -1, self.HEADS, self.HEAD_DIM).permute(0, 2, 1, 3)
    V = V.view(batch_size, -1, self.HEADS, self.HEAD_DIM).permute(0, 2, 1, 3)
    # Q: [batch_size, n_heads, query_len, head_dim]
    # K: [batch_size, n_heads, key_len, head_dim]
    # V: [batch_size, n_heads, value_len, head_dim]

    # attention energies, (Q*K^T/n)
    energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.SCALE
    # energy: [batch_size, n_heads, query_len, key_len]

    # masking
    if mask is not None:
      energy = energy.masked_fill(mask==0, -1e10)

    # attention score, softmax(Q*K^T/n)
    attention = torch.softmax(energy, dim=-1)
    # attention: [batch_size, n_heads, query_len, key_len]

    # softmax(Q*K^T/n)*V
    x = torch.matmul(self.dropout(attention), V)
    # x: [batch_size, n_heads, query_len, head_dim]

    x = x.permute(0, 2, 1, 3).contiguous()
    x = x.view(batch_size, -1, self.HIDDEN_DIM)
    # x: [batch_size, query_len, hidden_dim]
    x = self.fc_o(x)
    return x, attention


class PositionwiseFeedforwardLayer(nn.Module):
  def __init__(self, HIDDEN_DIM, PF_DIM, dropout_ratio):
    super().__init__()
    self.fc_1    = nn.Linear(HIDDEN_DIM, PF_DIM)
    self.fc_2    = nn.Linear(PF_DIM, HIDDEN_DIM)
    self.dropout = nn.Dropout(dropout_ratio)

  def forward(self, x):
    # x: [batch_size, seq_len, hidden_dim]

    x = self.dropout(torch.relu(self.fc_1(x)))
    # x: [batch_size, seq_len, pf_dim]

    x = self.fc_2(x)
    # x: [batch_size, seq_len, hidden_dim]
    return x

### token embedding & positional encoding

In [9]:
class TokenEmbedding(nn.Module):
  def __init__(self, VOCAB_SIZE, HIDDEN_DIM):
    super().__init__()
    self.HIDDEN_DIM = HIDDEN_DIM
    self.embedding  = nn.Embedding(VOCAB_SIZE, HIDDEN_DIM)
    self.SCALE      = torch.sqrt(torch.FloatTensor([HIDDEN_DIM])).to(DEVICE)

  def forward(self, src):
    return self.embedding(src) * self.SCALE

In [10]:
import math

class PositionalEncoding(nn.Module):
  def __init__(self, HIDDEN_DIM, dropout_ratio=0.1, max_length=5000):
    super().__init__()
    self.dropout = nn.Dropout(dropout_ratio)

    pos = torch.arange(max_length).unsqueeze(1)
    den = torch.exp(torch.arange(0, HIDDEN_DIM, 2) * (-math.log(10000) / HIDDEN_DIM))
    
    pos_embedding = torch.zeros(max_length, 1, HIDDEN_DIM)
    pos_embedding[:, 0, 0::2] = torch.sin(pos * den)
    pos_embedding[:, 0, 1::2] = torch.cos(pos * den)
    self.register_buffer('pos_embedding', pos_embedding)

  def forward(self, token_embedding):
    # x: [seq_len, batch_size, hidden_dim]
    token_embedding += self.pos_embedding[:token_embedding.size(0), :]
    return self.dropout(token_embedding)

### transformer encoder

In [11]:
class EncoderLayer(nn.Module):
  def __init__(self, HIDDEN_DIM, HEADS, PF_DIM, dropout_ratio):
    super().__init__()
    self.self_attn_layer_norm = nn.LayerNorm(HIDDEN_DIM)
    self.ff_layer_norm        = nn.LayerNorm(HIDDEN_DIM)

    self.self_attention           = MultiHeadAttentionLayer(HIDDEN_DIM, HEADS, dropout_ratio)
    self.positionwise_feedforward = PositionwiseFeedforwardLayer(HIDDEN_DIM, PF_DIM, dropout_ratio)
    self.dropout = nn.Dropout(dropout_ratio)

  def forward(self, src, src_mask):
    # src     : [batch_size, src_len, hidden_dim]
    # src_mask: [batch_size, src_len]

    # self attention
    _src, _ = self.self_attention(src, src, src, src_mask)

    # residual connection & layer norm
    src = self.self_attn_layer_norm(src + self.dropout(_src))

    # position-wise feed forward
    _src = self.positionwise_feedforward(src)

    # residual connection & layer norm
    src = self.ff_layer_norm(src + self.dropout(_src))
    return src


class Encoder(nn.Module):
  def __init__(self, INPUT_DIM, HIDDEN_DIM, LAYERS, HEADS, PF_DIM, dropout_ratio, max_length=100):
    super().__init__()
    self.src_token_embedding = TokenEmbedding(INPUT_DIM, HIDDEN_DIM)
    self.positional_encoding = PositionalEncoding(HIDDEN_DIM, dropout_ratio)
    self.layers = nn.ModuleList([EncoderLayer(HIDDEN_DIM, HEADS, PF_DIM, dropout_ratio)
                                for _ in range(LAYERS)])
    
    self.scale   = torch.sqrt(torch.FloatTensor([HIDDEN_DIM])).to(DEVICE)
    self.dropout = nn.Dropout(dropout_ratio)

  def forward(self, src, src_mask):
    # src     : [batch_size, src_len]
    # src_mask: [batch_size, src_len]

    src = self.src_token_embedding(src)
    src = self.positional_encoding(src)

    # forward for all layers
    for layer in self.layers:
      src = layer(src, src_mask)
    # src: [batch_size, src_len, hidden_dim]
    return src

### transformer decoder

In [12]:
class DecoderLayer(nn.Module):
  def __init__(self, HIDDEN_DIM, HEADS, PF_DIM, dropout_ratio):
    super().__init__()
    self.self_attn_layer_norm  = nn.LayerNorm(HIDDEN_DIM)
    self.enc_attn_layer_norm   = nn.LayerNorm(HIDDEN_DIM)
    self.ff_layer_norm         = nn.LayerNorm(HIDDEN_DIM)

    self.self_attention           = MultiHeadAttentionLayer(HIDDEN_DIM, HEADS, dropout_ratio)
    self.encoder_attention        = MultiHeadAttentionLayer(HIDDEN_DIM, HEADS, dropout_ratio)
    self.positionwise_feedforward = PositionwiseFeedforwardLayer(HIDDEN_DIM, PF_DIM, dropout_ratio)
    self.dropout = nn.Dropout(dropout_ratio)

  # attention for encoding's output
  def forward(self, trg, enc_src, trg_mask, src_mask):
    # trg     : [batch_size, trg_len, hidden_dim]
    # enc_src : [batch_size, src_len, hidden_dim]
    # trg_mask: [batch_size, trg_len]
    # src_mask: [batch_size, src_len]

    # self attention
    _trg, _ = self.self_attention(trg, trg, trg, trg_mask)

    # residual connection & layer norm
    trg = self.self_attn_layer_norm(trg + self.dropout(_trg))

    # encoder attention
    _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)

    # residual connection & layer_norm
    trg = self.enc_attn_layer_norm(trg + self.dropout(_trg)) 

    # positionwise feed forward
    _trg = self.positionwise_feedforward(trg)

    # residual connection & layer norm
    trg = self.ff_layer_norm(trg + self.dropout(_trg))

    # trg      : [batch_size, trg_len, hidden_dim]
    # attention: [batch_size, n_heads, trg_len, src_len]
    return trg, attention


class Decoder(nn.Module):
  def __init__(self, OUTPUT_DIM, HIDDEM_DIM, LAYERS, HEADS, PF_DIM, dropout_ratio, max_length=100):
    super().__init__()
    self.trg_token_embedding = TokenEmbedding(OUTPUT_DIM, HIDDEN_DIM)
    self.positional_encoding = PositionalEncoding(HIDDEN_DIM, dropout_ratio)
    self.layers = nn.ModuleList([DecoderLayer(HIDDEN_DIM, HEADS, PF_DIM, dropout_ratio)
                                for _ in range(LAYERS)])

    self.fc_out  = nn.Linear(HIDDEN_DIM, OUTPUT_DIM)
    self.scale   = torch.sqrt(torch.FloatTensor([HIDDEN_DIM])).to(DEVICE)
    self.dropout = nn.Dropout(dropout_ratio)

  def forward(self, trg, enc_src, trg_mask, src_mask):
    # trg     : [batch_size, trg_len]
    # enc_src : [batch_size, src_len, hidden_dim]
    # trg_mask: [batch_size, trg_len]
    # src_mask: [batch_size, src_len]

    trg = self.trg_token_embedding(trg)
    trg = self.positional_encoding(trg)

    for layer in self.layers:
      trg, attention = layer(trg, enc_src, trg_mask, src_mask)
    # trg      : [batch_size, trg_len, hidden_dim]
    # attention: [batch_size, n_heads, trg_len, src_len]

    output = self.fc_out(trg)
    # output: [batch_size, trg_len, output_dim]
    return output, attention

### transformer

In [13]:
class Transformer(nn.Module):
  def __init__(self, encoder, decoder, pad_idx):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_pad_idx, self.trg_pad_idx = pad_idx, pad_idx

  # masking for <pad> token
  def make_src_mask(self, src):
    # src: [batch_size, src_len]

    src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
    # src_mask: [batch_size, 1, 1, src_len]
    return src_mask

  # masking for next tokens in target sentence
  def make_trg_mask(self, trg):
    # trg: [batch_size, trg_len]

    trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
    # trg_pad_mask: [batch_size, 1, 1, trg_len]

    trg_len = trg.shape[1]
    trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device=DEVICE)).bool()
    # trg_sub_mask: [trg_len, trg_len]

    trg_mask = trg_pad_mask & trg_sub_mask
    # trg_mask: [batch_size, 1, trg_len, trg_len]
    return trg_mask

  def forward(self, src, trg):
    # src: [batch_size, src_len]
    # trg: [batch_size, trg_len]

    src_mask = self.make_src_mask(src)
    trg_mask = self.make_trg_mask(trg)
    # src_mask: [batch_size, 1, 1, src_len]
    # trg_mask: [batch_size, 1, trg_len, trg_len]

    enc_src = self.encoder(src, src_mask)
    # enc_src: [batch_size, src_len, hidden_dim]

    output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
    # output   : [batch_size, trg_len, output_dim]
    # attention: [batch_size, n_heads, trg_len, src_len]
    return output, attention

In [14]:
# hyper parameters
INPUT_DIM, OUTPUT_DIM = len(en_vocab), len(de_vocab)
HIDDEN_DIM = 512
LAYERS     = 6
HEADS      = 8
PF_DIM     = 2048
DROPOUT    = 0.1

In [15]:
encoder = Encoder(INPUT_DIM, HIDDEN_DIM, LAYERS, HEADS, PF_DIM, DROPOUT)
decoder = Decoder(OUTPUT_DIM, HIDDEN_DIM, LAYERS, HEADS, PF_DIM, DROPOUT)
model   = Transformer(encoder, decoder, preprocessor.pad_token_id)

def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainable parameters.\n')

def initialize_weights(model):
  for p in model.parameters():
    if p.dim() > 1: nn.init.xavier_uniform_(p)

model.apply(initialize_weights)
model = model.to(DEVICE)

The model has 55,522,638 trainable parameters.



In [16]:
import torch.optim as optim

# using adam optimizer
optimizer = torch.optim.Adam(model.parameters(),
                             lr=0.0001,
                             betas=(0.9, 0.98),
                             eps=1e-9) 

# define loss function, ignore for padding value
loss_fn = nn.CrossEntropyLoss(ignore_index=preprocessor.pad_token_id)

### training

In [17]:
def train(model, iterator, optimizer, loss_fn, clip):
  model.train()
  epoch_loss = 0

  for i, batch in enumerate(iterator):
    src, trg = batch[0], batch[1]
    
    optimizer.zero_grad()  # make gradients zero before backpropagation

    output, _ = model(src, trg[:,:-1])        # ignore for target's <eos>
    # output: [batch_size, trg_len-1, output_dim]
    # trg   : [batch_size, trg_len]

    output_dim = output.shape[-1]
    output = output.contiguous().view(-1, output_dim)
    trg    = trg[:,1:].contiguous().view(-1)  # ignore for target's <sos>
    # output: [batch_size * trg_len-1, output_dim]
    # trg   : [batch_size * trg_len-1]

    loss = loss_fn(output, trg)
    epoch_loss += loss.item()

    loss.backward()                                           # compute gradient
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)  # gradient clipping
    optimizer.step()                                          # update parameters
  return epoch_loss / len(iterator)

def evaluate(model, iterator, loss_fn):
  model.eval()
  epoch_loss = 0

  with torch.no_grad():
    for i, batch in enumerate(iterator):
      src, trg = batch[0], batch[1]

      output, _ = model(src, trg[:,:-1])        # ignore for target's <eos>
      # output: [batch_size, trg_len-1, output_dim]
      # trg   : [batch_size, trg_len]

      output_dim = output.shape[-1]
      output = output.contiguous().view(-1, output_dim)
      trg    = trg[:,1:].contiguous().view(-1)  # ignore for target's <sos>
      # output: [batch_size * trg_len-1, output_dim]
      # trg   : [batch_size * trg_len-1]

      loss = loss_fn(output, trg)
      epoch_loss += loss.item()
  return epoch_loss / len(iterator)

def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

In [18]:
import time

EPOCHS = 10
CLIP   = 1
best_valid_loss = float('inf')

for epoch in range(EPOCHS):
  start_time = time.time()

  train_loss = train(model, train_loader, optimizer, loss_fn, CLIP)
  valid_loss = evaluate(model, valid_loader, loss_fn)

  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), 'transformer_en_to_de.pt')

  print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
  print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')
  print('\n')

Epoch: 01 | Time: 1m 0s
	Train Loss: 5.412 | Train PPL: 224.045
	Validation Loss: 4.656 | Validation PPL: 105.235


Epoch: 02 | Time: 0m 59s
	Train Loss: 4.464 | Train PPL: 86.848
	Validation Loss: 4.105 | Validation PPL: 60.638


Epoch: 03 | Time: 1m 0s
	Train Loss: 3.913 | Train PPL: 50.064
	Validation Loss: 3.682 | Validation PPL: 39.743


Epoch: 04 | Time: 1m 1s
	Train Loss: 3.594 | Train PPL: 36.392
	Validation Loss: 3.424 | Validation PPL: 30.699


Epoch: 05 | Time: 1m 1s
	Train Loss: 3.363 | Train PPL: 28.881
	Validation Loss: 3.240 | Validation PPL: 25.522


Epoch: 06 | Time: 1m 1s
	Train Loss: 3.160 | Train PPL: 23.560
	Validation Loss: 3.063 | Validation PPL: 21.387


Epoch: 07 | Time: 1m 1s
	Train Loss: 2.979 | Train PPL: 19.672
	Validation Loss: 2.919 | Validation PPL: 18.515


Epoch: 08 | Time: 1m 1s
	Train Loss: 2.828 | Train PPL: 16.917
	Validation Loss: 2.819 | Validation PPL: 16.753


Epoch: 09 | Time: 1m 1s
	Train Loss: 2.698 | Train PPL: 14.846
	Validation Loss: 2.71

In [19]:
# download saved model
from google.colab import files
files.download('transformer_en_to_de.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### testing

In [20]:
# load saved model
model.load_state_dict(torch.load('./transformer_en_to_de.pt'))

# test
test_loss = evaluate(model, test_loader, loss_fn)
print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f}')

Test Loss: 2.627 | Test PPL: 13.836


In [21]:
# translation using my model
def translate_sentence(sentence, preprocessor, model, max_len=50):
  model.eval()

  # make source indices
  src_indices = preprocessor.src_encode(sentence)
  src_indices = [preprocessor.sos_token_id] + src_indices + [preprocessor.eos_token_id]

  # using encoder
  src_tensor = torch.LongTensor(src_indices).unsqueeze(0).to(DEVICE)
  src_mask   = model.make_src_mask(src_tensor)
  with torch.no_grad():
    enc_src = model.encoder(src_tensor, src_mask)

  # using decoder, make translated target indices
  trg_indices = [preprocessor.sos_token_id]            # has <sos> token
  for i in range(max_len):
    trg_tensor = torch.LongTensor(trg_indices).unsqueeze(0).to(DEVICE)
    trg_mask   = model.make_trg_mask(trg_tensor)
    with torch.no_grad():
      output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)

    pred_token = output.argmax(2)[:,-1].item()
    if pred_token == preprocessor.eos_token_id: break  # break when <eos> token
    else: trg_indices.append(pred_token)

  # convert indices to words
  trg_tokens = [preprocessor.tgt_id2token[id] for id in trg_indices]
  return trg_tokens[1:], attention                     # remove <sos> token

In [22]:
for i, batch in enumerate(test_dataset):
  if i == 10:
    src, tgt = batch[0], batch[1]
    break
src = preprocessor.src_decode(src)
tgt = preprocessor.tgt_decode(tgt)
print('original en sentence:')
print(f'\t{src}')
print('original de sentence:')
print(f'\t{tgt}')

translation, _ = translate_sentence(src, preprocessor, model)
translation = ' '.join(translation)
print('translated de sentence:')
print(f'\t{translation}')

original en sentence:
	A mother and her young song enjoying a beautiful day outside .
original de sentence:
	Eine Mutter und ihr kleiner Sohn genießen einen schönen Tag im Freien .
translated de sentence:
	Eine junge Frau sitzt auf einem <unk> und genießt ihre <unk> .


In [23]:
# for bleu score
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 1.2 MB/s 
     |████████████████████████████████| 1.3 MB 42.9 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1


In [24]:
from torchtext.data.metrics import bleu_score

def show_bleu(data, preprocessor, model, max_len=50):
  trgs, pred_trgs = [], []
  for i in range(len(data)):
    # original target
    src, trg = data[i][0], data[i][1]
    src = preprocessor.src_decode(src)
    trg = preprocessor.tgt_decode(trg).split(' ')
    trgs.append([trg])

    # predicted target
    pred_trg, _ = translate_sentence(src, preprocessor, model, max_len)
    pred_trg = pred_trg[:-1]
    pred_trgs.append(pred_trg)

    if (i + 1)%100 == 0:
      print(f"[{i+1}/{len(data)}]")
      print(f"정답: {trg}")
      print(f"예측: {pred_trg}\n")

  bleu = bleu_score(pred_trgs, trgs, max_n=4, weights=[0.25, 0.25, 0.25, 0.25])
  print(f'Total BLEU Score = {bleu*100:.2f}')

  bleu1_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[1, 0, 0, 0])
  bleu2_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[0, 1, 0, 0])
  bleu3_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[0, 0, 1, 0])
  bleu4_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[0, 0, 0, 1])
  print(f'BLEU1 score = {bleu1_score}') 
  print(f'BLEU2 score = {bleu2_score}') 
  print(f'BLEU3 score = {bleu3_score}') 
  print(f'BLEU4 score = {bleu4_score}') 

show_bleu(test_dataset, preprocessor, model)

[100/1000]
정답: ['Ein', 'kleiner', 'Junge', 'im', 'Fußballdress', 'hält', 'die', 'Hände', '<unk>', 'Gesicht', 'und', 'weint', '.']
예측: ['Ein', 'kleiner', 'Junge', 'in', 'einem', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']

[200/1000]
정답: ['Ein', 'Mann', 'macht', 'Werbung', 'mit', 'einem', 'riesigen', 'Schild', ',', 'das', 'auf', 'sein', 'Fahrrad', 'gebunden', 'ist', '.']
예측: ['Ein', 'Mann', 'mit', 'einem', '<unk>', '<unk>', '<unk>', 'auf', 'einem', '<unk>']

[300/1000]
정답: ['Eine', 'Gruppe', 'junger', 'Menschen', 'trinkt', '<unk>', 'in', 'einem', '<unk>', '<unk>', '.']
예측: ['Eine', 'Gruppe', 'junger', 'junger', 'junger', 'Menschen', 'macht', 'eine', '<unk>', 'in', 'der', 'Nähe', 'eines', '<unk>']

[400/1000]
정답: ['Ein', 'lächelnder', 'Junge', 'spielt', 'im', 'Laub', 'mit', 'den', 'Enten', '.']
예측: ['Ein', 'kleiner', 'Junge', 'spielt', 'im', 'Freien', 'mit', '<unk>']

[500/1000]
정답: ['Eine', 'Frau', 'steht', 'auf', 'einem', 'grünen',